# Machine Learning for Everybody – Full Course

https://youtu.be/i_LwzRVP7bg


# FCC Bikes Regression

### Seoul Bike Sharing Demand Data Set

Data Source :http://data.seoul.go.kr/
SOUTH KOREA PUBLIC HOLIDAYS. URL: publicholidays.go.kr

https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand

In [1]:
!curl --create-dir -O --output-dir ./data https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  590k  100  590k    0     0   203k      0  0:00:02  0:00:02 --:--:--  204k01 --:--:--     0
